In [ ]:
from typing import Union, List

import re
import json
import subprocess
from pathlib import Path

import pandas as pd
import xlwings as xw


In [2]:
class ProjectException(Exception):
    '''Base Exception for projects module.'''
    pass


class AnacondaException(ProjectException):
    '''Errors related to `conda` calls.'''
    pass


## Utility Functions

### Console command processing

In [3]:
def error_check(output: subprocess.CompletedProcess, error_type: Exception,
                error_msg: str):
    '''Check for errors in subprocess output.

    if a `CalledProcessError` error occured raise the error, with the
    error_msg text.
    Args:
        output (subprocess.CompletedProcess): subprocess output to be tested.
        error_type (Exception): The type of exception to raise if an error
            occurs.
        error_msg (str): The Error message to include if required.
    Raises:
        error_type: If subprocess generated an error.
    '''
    try:
        output.check_returncode()
    except subprocess.CalledProcessError as err:
        msg = '\n'.join([error_msg, output.stderr.decode()])
        raise AnacondaException(msg) from err


In [ ]:
def console_command(cmd: Union[str, List[str]], error_type: Exception,
                    error_msg: str) -> str:
    '''Run a system console command.

    Run the command. Check for errors and raise the appropriate error if
    necessary.
    If no error, return the output from the result of running the console
    command.

    Args:
        cmd (str or list): The console command to execute.
        error_type (Exception): The type of exception to raise if an error
            occurs.
        error_msg (str): The Error message to include if required.

    Returns:
        str: The log output from running the console command.
    '''
    if isinstance(cmd, str):
        cmd_list = cmd.split(' ')
    else:
        cmd_list = cmd
    output = subprocess.run(cmd_list, shell=True, capture_output=True)
    # check for errors
    error_check(output, error_type, error_msg)
    cmd_log = output.stdout.decode()
    return cmd_log


In [5]:
def save_env_specs(env_name: str, env_folder: Path):
    '''Store *spec* and *.yml* for the environment.

    Args:
        new_env (str): Name of Conda environment.
        env_folder (str): Path to the project's environment folder.
    '''
    env_dict = list_environments()
    env_path = env_dict[env_name]

    # Save spec file
    spec_file = env_folder / f'{env_name}_specfile.txt'
    save_spec_cmd = ['conda', 'list', '--explicit', '-p', env_path]
    output = subprocess.run(save_spec_cmd, capture_output=True, shell=True)
    error_check(output, AnacondaException,
                f'Error saving {env_name} environment spec file.')
    with open(spec_file, 'w') as f:
        f.write(output.stdout.decode())

    # Save full yml file
    full_yml_file = env_folder / f'{env_name}_FULL.yml'
    save_full_yml_cmd = ['conda', 'env', 'export', '-p', env_path, '--file', str(full_yml_file)]
    output = subprocess.run(save_full_yml_cmd, capture_output=True, shell=True)
    error_check(output, AnacondaException,
                f'Error saving {env_name} full .yml file.')

    # Save history json file
    history_yml_file = env_folder / f'{env_name}.json'
    save_yml_cmd = ['conda', 'env', 'export', '--from-history', '--json', '-p', env_path]
    output = subprocess.run(save_yml_cmd, capture_output=True, shell=True)
    error_check(output, AnacondaException,
                f'Error saving {env_name} history .json to {str(history_yml_file)}.')
    with open(history_yml_file, 'w') as f:
        f.write(output.stdout.decode())


In [6]:
def get_conda_config(conda_info_file: Path = None)->dict:
    '''Get information about the current Conda environment.

    If a file path is provided, save the json info to that file.

    Args:
        conda_info_file (Path, optional): The file path to save the json data
            in. If None, do not save the data.  Defaults to None.

    Returns:
        dict: Conda environment parameters as nested dictionaries.
    '''
    conda_info_cmd = r'conda info --envs --json'
    conda_info = console_command(conda_info_cmd, AnacondaException,
                                 'Unable to get Anaconda info')

    # If supplied, save the data to a .json file
    if conda_info_file:
        conda_info_file.write_text(conda_info)

    # Convert the json data to a dictionary
    conda_info_dict = json.loads(conda_info)
    return conda_info_dict


In [7]:
def list_environments():
    env_pattern = re.compile(
        r'(?P<name>'  # Start of *name* group.
        r'[a-z0-9_]'  # Name begins with letter, number or _.
        r'.+?'        # All text until 2 or more spaces are encountered
        r')'          # End of *name* group.
        r'[ *]{2,}'   # 2 or more spaces or * in a row.
        r'(?P<path>'  # Start of *path* group.
        r'[A-Z]:'     # Drive letter, followed by a :.
        r'[^\r\n]*'   # Remaining text before the end of the line.
        r')',         # End of *path* group.
        flags=re.IGNORECASE)

    conda_envs = r'conda env list'
    env_list = console_command(conda_envs, AnacondaException,
                               'Unable to get Anaconda environments')
    env_info = {name: path for name, path in env_pattern.findall(env_list)}
    return env_info


- Set the path to store the conda info in

In [8]:
env_storage_path = Path(r'C:\temp\envs')


In [9]:
if not env_storage_path.exists():
    env_storage_path.mkdir()


- Get list of current Anaconda environments

In [10]:
env_dict = list_environments()
env_names = list(env_dict.keys())


- Store environment info for each environment

**FIXME:  This does not work for environment with spaces in their names**

In [11]:
for this_env in env_names:
    print(f'Storing environment for {this_env}')
    try:
        save_env_specs(this_env, env_storage_path)
    except AnacondaException:
        print(f'Unable to store environment for {this_env}')


Storing environment for base
Storing environment for DCF_Tracking
Storing environment for DCF_Tracking
Storing environment for DICOM_Burn
Storing environment for DICOM_Burn
Storing environment for DICOM_Repair
Storing environment for DICOM_Repair
Storing environment for DICOM_Repair_dev
Storing environment for DICOM_Repair_dev
Storing environment for DirScanCompile
Storing environment for DirScanCompile
Storing environment for DocumentTests
Storing environment for DocumentTests
Storing environment for EDW_QA
Storing environment for EDW_QA
Storing environment for Electrons
Storing environment for Electrons
Storing environment for Form Extraction
Storing environment for Form Extraction
Storing environment for FormExtraction
Storing environment for FormExtraction
Storing environment for Images
Storing environment for Images
Storing environment for Limbus
Storing environment for Limbus
Storing environment for Local_xlwings
Storing environment for Local_xlwings
Storing environment for McMed

- Manual mode for environments with spaces

- Save table with environments and their paths

In [15]:
env_data = pd.DataFrame([env_dict]).T.reset_index()
env_data.columns = ['Environment', 'Environment Path']

env_table_file = env_storage_path / 'Conda Environments.xlsx'
env_data.to_excel(env_table_file)


- Save json file with conda configuration data

In [16]:
conda_info_file = env_storage_path / 'conda_info.json'
conda_info = get_conda_config(conda_info_file)